<a href="https://colab.research.google.com/github/SMotaee/Deep-Learning/blob/main/Detection%20of%20driver%20tiredness%20by%20counting%20eye%20blinks%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('eye_blink_detection.csv')

# Split the dataset into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Load the images and labels for the training set
train_images = []
train_labels = []
for index, row in train.iterrows():
    image = cv2.imread(row['image_path'])
    image = cv2.resize(image, (32, 32))
    train_images.append(image)
    train_labels.append(row['label'])

# Load the images and labels for the testing set
test_images = []
test_labels = []
for index, row in test.iterrows():
    image = cv2.imread(row['image_path'])
    image = cv2.resize(image, (32, 32))
    test_images.append(image)
    test_labels.append(row['label'])

# Convert the images and labels to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize the pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the neural network model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
model.save('blink_detection_model.h5')


In [ ]:
import cv2
import dlib
import numpy as np
from tensorflow.keras.models import load_model

# Load the eye blink detection model
model = load_model('blink_detection_model.h5')

# Initialize the dlib face detector and the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define a function to calculate the eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the euclidean distances between the two sets of vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])

    # Calculate the euclidean distance between the horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])

    # Calculate the EAR
    ear = (A + B) / (2.0 * C)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the trained model
model = load_model('blink_detection_model.h5')

# Load the dataset
data = pd.read_csv('eye_blink_detection.csv')

# Load the images and labels
images = []
labels = []
for index, row in data.iterrows():
    image = cv2.imread(row['image_path'])
    image = cv2.resize(image, (32, 32))
    images.append(image)
    labels.append(row['label'])

# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the pixel values to be between 0 and 1
images = images / 255.0

# Evaluate the model on the dataset
predictions = model.predict(images)
predictions = np.round(predictions).flatten()
accuracy = accuracy_score(labels, predictions)

print("Accuracy: {:.2f}%".format(accuracy * 100))